In [1]:
import os
from nltk import word_tokenize
from collections import Counter
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer



data = {}

#reading the folder for dataset i.e. files and folders and storing in a dictionary
for root, dirs, files in os.walk("20_newsgroups"): 
    for d in dirs:
        data[d]=[]
        for f in (os.scandir("20_newsgroups//"+d)):
            data[d].append(f.name)
            
train ={}
test ={}

for d in data.keys():    
    l = int(len(data[d])*0.5)    
    train[d] = data[d][0:l]
    test[d] = data[d][l:2*l]

words =[]
tr_words ={}    
words_count = {}
spl_characters = ['~','`','!','@','^','$','%','&','*','(',')','+','=','{','}','[',']',';',':','|','\\','"',"'",'\n','<','>',',','.','?','/','-','*']

#training the model with half of the data from dataset
print("Reading files from following directories:")
for i in train:  
    tr_words[i]=[]
    for j in train[i]:
        with open(("20_newsgroups//"+i +"//"+j),encoding="utf-8", errors='ignore') as value:
            value = value.read()
            value = value.replace("\n"," ")
            for k in spl_characters:
                value.replace(k," ")    
            tr_words[i].extend(word_tokenize(value))
    words_count[i] = Counter(tr_words[i])
    words.extend(tr_words[i])  
    print(i)
words = set(words)


print('The total number of words found in the dataset are: {}'.format(len(words)))

print("Training the model with the dataset.........")
words = list(words)
dictionary = {}
doc_num = 1
for i in words: 
    dictionary[i] = {}
    for j in tr_words: 
        if i in words_count[j]:
            dictionary[i][j] = words_count[j][i]
        else:
            dictionary[i][j] = 0.1
        dictionary[i][j]/=len(tr_words[j])
    
prob = {}
total= 0

# Probabilities to calcluate the accuracy of the model
print("Calculating probabilities of words......")
for i in train:
    prob[i] = len(train[i])
    total += len(train[i])
for j in prob:
    prob[j] = prob[j]/total

print("Model is now ready for testing.")

stemmer = PorterStemmer()
doc_prob = {}
acc = 0
c = 0

#testing the data with the reamining half of the data
print("Beginning the testing part of the model with calculated probabilities.....")
for d in test:
    for j in test[d]:
        
        with open(("20_newsgroups//"+d +"//"+j),encoding="utf-8", errors='ignore') as value:
            value = value.read()
            test_words = word_tokenize(value)
            test_words = set(test_words)
            p = {}
            for k in test:
                p[k] = prob[k]
                for w in test_words:
                    if w in dictionary:
                        p[k] = p[k]* dictionary[w][k]*1000
                    else:
                        p[k] = p[k]* 0.1/len(tr_words[d])*1000
            max_keys = [k for k, v in p.items() if v == max(p.values())]
            acc += (max_keys[0]==d)
            c+=1

print("Calculating accuracy.......")         
print("Accuracy: ", acc*100/c)
    
    


    
        


Reading files from following directories:
alt.atheism
comp.graphics
comp.os.ms-windows.misc
comp.sys.ibm.pc.hardware
comp.sys.mac.hardware
comp.windows.x
misc.forsale
rec.autos
rec.motorcycles
rec.sport.baseball
rec.sport.hockey
sci.crypt
sci.electronics
sci.med
sci.space
soc.religion.christian
talk.politics.guns
talk.politics.mideast
talk.politics.misc
talk.religion.misc
The total number of words found in the dataset are: 189809
Training the model with the dataset.........
Calculating probabilities of words......
Model is now ready for testing.
Beginning the testing part of the model with calculated probabilities.....
Calculating accuracy.......
Accuracy:  76.90538107621525
